In [1]:
# Radimo primer genetskog programiranja gde resavamo neki matematicki izraz, tj hocemo da napravimo neko stablo
# koje kada se evaluira daje neki broj, npr 100. Ovo samo po sebi je prelako, jer moze samo da se napravi jedan koreni cvor kloji u sebi ima 100
# tako da uvodimo ogranicenja:
#     hocemo da bude sastavljeno samo od brojeva iz intervala [-10, 10]
#     i da stablo uvek ima dubinu 3 (da bude binarno stablo sa 4 lista, tj 7 cvorova ukupno (1    2   4))
#     (ovo ce se evaluirati kao      (a op1 b) op2 (c op3 d)         (a,b,c,d su listovi)    )

# za unutrasnje cvorove mozemo da imamo +,-,*,/ (ovo zovemo primitivne funkcije)
# listovi ce nam biti neki brojevi   (ovo zovemo terminali)
import random

In [2]:
# NOTE: mogli smo da pravimo da jedinke bas budu stabla, ali posto je primer koji radimo
# jednostavan, mozemo da se izvucemo i sa tim da nemamo pravo stablo, nego da kao genetski kod cuvamo listu
# operatora i brojeva
class Individual:
#     # ovako bismo npr mogli da napravimo zapravo stablo
#     def __init__(self, data, left, right):
#         self.data = data
#         self.left = left
#         self.right = right

    def __init__(self, goal=100, num_operators=3, num_terminals=4):
        # kod ce biti oblika [op1, op2, op3, a, b, c, d]  ,random inicijalizovan
        self.numOperators = num_operators
        self.numTerminals = num_terminals
        self.goal = goal
        self.code = []
        for i in range(num_operators):
            self.code.append(self.randomOperator())
        for i in range(num_terminals):
            self.code.append(self.randomTerminal())
        self.fitness = self.calcFitness()
        
    def __str__(self):
        a = str(self.code[3])
        b = str(self.code[4])
        c = str(self.code[5])
        d = str(self.code[6])
        first = '(' + a + ' ' + self.code[1] + ' ' + b + ')' 
        second = '(' + c + ' ' + self.code[2] + ' ' + d + ')'
        return  first + ' ' + self.code[0] + ' ' + second
    
    def __lt__(self, other):
        return self.fitness < other.fitness
            
    def randomOperator(self):
        operators = ['+', '-', '*']   # NOTE: izbacili smo deljenje da nebi slucajno negde podelili nulom, ali mozemo i bez
                                     # deljenja, pritom smo zapravo jos zakomplikovali zadatak
        return random.choice(operators)
    
    def randomTerminal(self, lb=-10, ub=10):
        return random.randrange(lb, ub+1)    # vraca neki ceo broj iz intervala
    
    def calcFitness(self):
        # za fitnes hocemo da evaluiramo stablo i da vratimo razliku dobijenog rezultata i 100 (taj broj trazimo)
        # u pajtonu psotoji funkcija eval() kojoj se prosledi neki string izraz npr eval('2+3') i ona to izracuna i vrati kao broj
        # mi smo ovde za operatore koristili karaktere pa nam je zgodno da korstimo eval
        # iznad smo overloadovali i konverziju jedinke u string
        value = eval(str(self))
        # minus ispred da bi se fitness poklapao sa svojim znacenjem u obicnom genetskom alg
        # sto je broj blizi 100 to je fitness veci
        return -abs(self.goal - value)
        

In [3]:
# i = Individual()
# print(i)

# print(i.fitness)
# print(eval(str(i)))

In [4]:
# nadalje samo radimo obican genetski alg

In [5]:
def selection(population):
    TOURNAMENT_SIZE = 5
    winner = max(random.sample(population, TOURNAMENT_SIZE))
    return winner

In [6]:
def getSubtree(rootIndex, subtreeIndices, totalNumNodes):
    # [op1, op2, op3, a, b, c, d]
    #  0    1    2    3  4  5  6
    # mozemo ovo da uradimo sa 3-4 if-a za ovaj konkretan primer jer je mali, ali za primer cemo da 
    # kodiramo tako da ova funkc radi i za malo opstiji problem, tj vece stablo
    # znamo da za potpuno binarno stablo predstavljeno preko liste, deca nekog cvora i se nalaze na 2*i+1
    # i na 2*i+2
    if rootIndex > totalNumNodes:
        return
    subtreeIndices.append(rootIndex)
    getSubtree(rootIndex * 2 + 1, subtreeIndices, totalNumNodes)
    getSubtree(rootIndex * 2 + 2, subtreeIndices, totalNumNodes)
    

In [7]:
# crossover cemo da radimo zamenom 'podstabla'. Ako se odabere koren pri ukrstanju, to nema mnogo smisla
# jer ce samo parent1 i parent2 zameniti mesta i postati deca, tako da ne razmatramo indeks 1
def crossover(parent1, parent2, child1, child2):
    rootIndex = random.randrange(1, len(parent1.code)) 
    subtreeIndices = []
    getSubtree(rootIndex, subtreeIndices, len(parent1.code))
    for i in range(len(parent1.code)):
        # cvorove iz podstabla menjamo a one koji nisu u podstablu ostavljamo iste
        if i in subtreeIndices:
            child1.code[i] = parent2.code[i]
            child2.code[i] = parent1.code[i]
        else: 
            child1.code[i] = parent1.code[i]
            child2.code[i] = parent2.code[i]
            

In [8]:
def mutation(individual):
    MUTATION_PROB = 0.05
    for i in range(len(individual.code)):
        if random.random() < MUTATION_PROB:
            if i < individual.numOperators:
                individual.code[i] = individual.randomOperator()
            else:
                individual.code[i] = individual.randomTerminal()

In [26]:
POPULATION_SIZE = 100
NUM_GENERATIONS = 2000
ELITISM_SIZE = POPULATION_SIZE // 3
# NOTE: neparan broj za elitism size bi nam u ptelji ispod bacao gresku za index out of range, zato menjamo da uvek bude paran
if ELITISM_SIZE % 2 == 1:
    ELITISM_SIZE += 1
    
# ovaj genetski alg nece npr moci da nadje resenje ako je goal neki prost broj van dozvoljenog intervala za terminale
# nece moci ni ako je broj prevelik/premali, mozemo da eksperimentisemo sa tim parametrima
# jedan nacin da se pretraga olaksa, je da se tokom iteriranja povecava interval za terminale
population = [Individual(goal=100) for _ in range(POPULATION_SIZE)]
newPopulation = [Individual(goal=100) for _ in range(POPULATION_SIZE)]

for i in range(NUM_GENERATIONS):
    population.sort(reverse=True)
    
    # generalno pustamo genetski alg da se vrti NUM_GENERATIONS puta kada ne znamo sta je tacno resenje koje trazimo
    # a ovde ako znamo mozemo i ranije da izadjemo iz petlje
    if population[0].fitness == 0:
        break
        
    newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
    for j in range(ELITISM_SIZE, POPULATION_SIZE, 2):   # ne zaboravi ovde sa korakom 2 zbog nacina kako radimo sa child
        parent1 = selection(population)
        parent2 = selection(population)
            
        crossover(parent1, parent2, newPopulation[j], newPopulation[j+1]) 
            
        mutation(newPopulation[j])
        mutation(newPopulation[j+1])
        
        newPopulation[j].fitness = newPopulation[j].calcFitness();
        newPopulation[j+1].fitness = newPopulation[j+1].calcFitness();
            
    population = newPopulation
    
best = max(population)

print(f'solution: {best}  fitness: {best.fitness}')

solution: (-9 * -8) * (-5 - -9)  fitness: -1
